In [1]:
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id_akun = "hacktiv8-408704" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id_akun)

Authenticated


In [3]:
import pandas as pd

In [6]:
df = client.query('''
SELECT *
FROM `ftds-hacktiv8-project.phase0_ftds_012_hck.beverage_sales`
''').to_dataframe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6014 entries, 0 to 6013
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   invoice_id             6014 non-null   object 
 1   date                   6014 non-null   dbdate 
 2   store_id               6014 non-null   Int64  
 3   store_name             6014 non-null   object 
 4   store_city             6014 non-null   object 
 5   product_name           6014 non-null   object 
 6   manufacturing_company  6014 non-null   object 
 7   pack                   6014 non-null   Int64  
 8   unit_volume_ml         6014 non-null   Int64  
 9   unit_sold              6014 non-null   Int64  
 10  unit_retail_price      6014 non-null   float64
 11  total_price            6014 non-null   float64
 12  volume_sold_liters     6014 non-null   float64
dtypes: Int64(4), dbdate(1), float64(3), object(5)
memory usage: 634.4+ KB


# Problem 1 - SQL

## 1. Kota apa saja yang masuk ke dalam cakupan wilayah distribusi?

In [9]:
storeCities = df['store_city'].unique()
print(len(storeCities))
storeCities

6


array(['CLARINDA', 'MUSCATINE', 'MASON CITY', 'WEST DES MOINES',
       'CLINTON', 'CEDAR FALLS'], dtype=object)

Kota yang masuk dalam cangkupan wilayah distribusi ada 6, yaitu Clarinda, Muscatine, Mason City, West Des Moines, Clinton, dan Cedar Falls

## 2. Berapa rentang harga minimum dan maksimum dari unit minumam yang dijual?

In [13]:
df = client.query('''
SELECT
MAX(unit_retail_price) as MaxPrice,
MIN(unit_retail_price) as MinPrice
FROM `ftds-hacktiv8-project.phase0_ftds_012_hck.beverage_sales`
''').to_dataframe()

df

,MaxPrice,MinPrice
0,54.0,0.98


Diketahui rentang harga minimum dari unit minuman yang dijual adalah 0.98 dan harga maksimum dari unit minuman yang dijual adalah 54

## 3. Bandingkan total penjualan dan jumlah transaksi dari masing-masing kota pada tahun 2022?

In [31]:
df = client.query('''
SELECT
store_city,
EXTRACT (YEAR FROM date) as years,
COUNT(*) as total_penjualan
FROM `ftds-hacktiv8-project.phase0_ftds_012_hck.beverage_sales`
WHERE EXTRACT (YEAR FROM date) = 2022
GROUP BY store_city, years
''').to_dataframe()

df

,store_city,years,total_penjualan
0,CLARINDA,2022,248
1,MUSCATINE,2022,234
2,MASON CITY,2022,47
3,WEST DES MOINES,2022,47
4,CLINTON,2022,387
5,CEDAR FALLS,2022,417


Store City Clarinda mendapatkan 248 total penjualan pada tahun 2022  
Store City Muscatine mendapatkan 234 total penjualan pada tahun 2022  
Store City Mason City mendapatkan 47 total penjualan pada tahun 2022  
Store City West Des Moines mendapatkan 47 total penjualan pada tahun 2022  
Store City Clinton mendapatkan 387 total penjualan pada tahun 2022  
dan Store City Cedar Falls mendapatkan 417 total penjualan pada tahun 2022

## 4. Produk apa yang paling banyak diminati pada tahun 2022?

In [32]:
df = client.query('''
SELECT
product_name,
EXTRACT (YEAR FROM date) as years,
COUNT(*) as count
FROM `ftds-hacktiv8-project.phase0_ftds_012_hck.beverage_sales`
WHERE EXTRACT (YEAR FROM date) = 2022
GROUP BY product_name, years
ORDER BY count DESC
limit 3
''').to_dataframe()

df

,product_name,years,count
0,FruitFusion Quencher,2022,291
1,Sparkling Bliss Brew,2022,120
2,VitaWave Splash,2022,111


Produk minuman yang paling banyak diminati di tahun 2022 adalah FruitFusion Quencher	dengan total 291

# Problem 2 - Pandas

In [33]:
import pandas as pd

## 1. Buat kolom baru, menunjukkan representasi musim dengan ketentuan berikut:
  |Musim|Bulan|
  |--|--|
  |Spring|Maret, April, Mei|
  |Summer|Juni, Juli, Agustus|
  |Autumn|September, Oktober, November|
  |Winter|Desember, Januari, Februari|

In [39]:
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id_akun = "hacktiv8-408704" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id_akun)

df = client.query('''
SELECT *
FROM `ftds-hacktiv8-project.phase0_ftds_012_hck.beverage_sales`
''').to_dataframe()
# mengganti format date menjadi datetime
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')

# function untuk mengubah bulan ke season
def month_to_season(month):
  if month in [3,4,5]:
    return 'Spring'
  elif month in [6,7,8]:
    return 'Summer'
  elif month in [9,10,11]:
    return 'Autumn'
  else:
    return 'Winter'

# extract bulan dari column date
df['month'] = df['date'].dt.month
# buat new column 'Season' dengan menerapkan function 'month'
df['Season'] = df['month'].apply(month_to_season)

df

Authenticated


,invoice_id,date,store_id,store_name,store_city,product_name,manufacturing_company,pack,unit_volume_ml,unit_sold,unit_retail_price,total_price,volume_sold_liters,month,Season
0,INV-58839200013,2023-05-26,6016,Metro Mall,CLARINDA,OasisGlow Refresher,NibbleNation Group,12,50,4,1.25,5.00,0.20,5,Spring
1,INV-56953900014,2023-03-24,6016,Metro Mall,CLARINDA,PureElixir Zest,MunchWave Snacks,12,50,5,1.30,6.50,0.25,3,Spring
2,INV-56345300016,2023-03-03,6016,Metro Mall,CLARINDA,Sparkling Bliss Brew,BiteBlend Delights,12,750,5,1.55,7.75,3.75,3,Spring
3,INV-32274100025,2020-11-27,6016,Metro Mall,CLARINDA,VelvetChai Charm,OliverCraft Inc.,12,750,6,1.75,10.50,4.50,11,Autumn
4,INV-32274100030,2020-11-27,6016,Metro Mall,CLARINDA,FruitFusion Quencher,CrunchCraft Co.,24,375,24,1.22,12.00,9.00,11,Autumn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6009,INV-46502900008,2022-04-14,4119,Peak Provisions,CEDAR FALLS,CocoCrave Delight,TasteTwist Enterprises,12,750,12,6.07,72.84,9.00,4,Spring
6010,INV-35080700016,2021-03-17,4119,Peak Provisions,CEDAR FALLS,CocoCrave Delight,TasteTwist Enterprises,12,750,24,6.07,145.68,18.00,3,Spring
6011,INV-20939700005,2019-07-30,4119,Peak Provisions,CEDAR FALLS,FruitFusion Quencher,CrunchCraft Co.,6,1750,18,1.94,34.92,31.50,7,Summer
6012,INV-50461000007,2022-08-18,4119,Peak Provisions,CEDAR FALLS,FrostedCocoa Savor,OliverCraft Inc.,12,750,12,3.91,46.92,9.00,8,Summer


Penambahan column Season dengan kriteria dimana:  
Jika transaksi dilakukan di bulan 3, 4, 5 (Maret, April, Mei) akan masuk ke Season Spring.  
Jika transaksi dilakukan di bulan 6, 7, 8 (Juni, Juli, Agustus) akan masuk ke Season Summer.  
Jika transaksi dilakukan di bulan 9, 10, 11 (September, Oktober, November) akan masuk ke Season Autumn.  
Jika transaksi dilakukan di bulan 12, 1, 2 (Desember, Januari, Februari) akan masuk ke Season Winter.

## 2. Berapa total barang yang terjual kepada toko Peak Provisions sepanjang tahun 2022? (Gunakan pandas query untuk menjawab soal no. 2)

In [41]:
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id_akun = "hacktiv8-408704" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id_akun)

df = client.query('''
SELECT *
FROM `ftds-hacktiv8-project.phase0_ftds_012_hck.beverage_sales`
''').to_dataframe()

# mengganti format date menjadi datetime
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')

# extract tahun dari column date
df['year'] = df['date'].dt.year

# mengambil data dari 'store_name' dengan nama toko 'Peak Provisions' di tahun 2022
df_total_sales = df[(df['store_name'] == 'Peak Provisions') & (df['year'] == 2022)]
total_unit_sold = df_total_sales['unit_sold'].sum()

total_unit_sold

Authenticated


4592

Total barang yang terjual kepada toko 'Peak Provisions' sepanjang tahun 2022 adalah 4592.

# Kesimpulan Akhir

Dengan 6 kota distribusi yang berada di Clarinda, Muscatine, Mason City, West Des Moines, Clinton, dan Cedar Falls memiliki harga minuman minimal 0.98 dan maksimum 54. Di tahun 2022, kota Cedar Falls berhasil mendapatkan total penjualan terbanyak dengan 417 total penjualan. Produk minuman yang paling banyak diminati di tahun 2022 adalah FruitFusion Quencher dengan total penjualan 291 unit. Total barang yang terjual kepada toko Peak Provisions sepanjang tahun 2022 adalah 4592 unit.